# Universidad Distrital Francisco José de Caldas
## Class: Advanced Image Processing - Remote Sensing with Python (Basic Activities)
### Por: David Alonso Rueda Rodríguez

### 2. Crear Imágenes Stack

#### Objetivo de la actividad:
- Crear layer stacked para la imagen Landsat RGB+NIR.

#### Librerías requeridas:
- **rasterio** (librería para comparación)
- **gdal** (librería para comparación)
- **earthpy** (librería para comparación)
- datetime

#### Fuente de datos:

https://gis.stackexchange.com/questions/223910/using-rasterio-or-gdal-to-stack-multiple-bands-without-using-subprocess-commands

In [26]:
import rasterio as ras
import glob
import json
import os
import re
import earthpy as et
import earthpy.spatial as es
from osgeo import gdal
from datetime import datetime

In [2]:
def time():
    from datetime import datetime as dt
    now = dt.now()
    timestamp = dt.timestamp(now)
    return timestamp

## Preparación de datos Landsat

### Determinación de Resolución espacial para bandas

Es necesario hacer una identificación de la resolución espacial y con este dato hacer la concatenacióno de las imágenes (layer stack), en caso de realizar la concatenación con todas las bandas estás serán remuestreadas a la menor resolución espacial disponible.

#### Obtener datos desde directorio de trabajo

In [4]:
# Obtener archivos jp2 en el directorio de interés
url_base = 'src/Landsat/'
images = glob.glob(url_base + '*.TIF')
images

['src/Landsat/LC08_L1TP_009057_20190903_20190917_01_T1_B2.TIF',
 'src/Landsat/LC08_L1TP_009057_20190903_20190917_01_T1_B3.TIF',
 'src/Landsat/LC08_L1TP_009057_20190903_20190917_01_T1_B4.TIF',
 'src/Landsat/LC08_L1TP_009057_20190903_20190917_01_T1_B5.TIF']

#### Identificación de resolución espacial para las bandas de la imagen satelital

In [5]:
# Resolución espacial de las bandas en la imagen satelital
bandas = []
res = []
print('Nombre Arhivo'+ '\t'*3 + 'Res. Espacial')
for image in images:
    with ras.open(image) as src:
        a1 = os.path.splitext(os.path.basename(image))[0]
        print('{}\t\t{:.0f}' .format(a1, src.profile['transform'][0], (-1)*src.profile['transform'][4]))
        bandas.append(a1)
        res.append(str(src.profile['transform'][0]) + ', ' + str(src.profile['transform'][2]))

Nombre Arhivo			Res. Espacial
LC08_L1TP_009057_20190903_20190917_01_T1_B2		30
LC08_L1TP_009057_20190903_20190917_01_T1_B3		30
LC08_L1TP_009057_20190903_20190917_01_T1_B4		30
LC08_L1TP_009057_20190903_20190917_01_T1_B5		30


## Layer Stack

### Layer Stack con Earthpy

Esta prueba permite identificar el tiempo empleado en la creación del layer stack con la librería **Earthpy**

In [15]:
images

['src/Landsat/LC08_L1TP_009057_20190903_20190917_01_T1_B2.TIF',
 'src/Landsat/LC08_L1TP_009057_20190903_20190917_01_T1_B3.TIF',
 'src/Landsat/LC08_L1TP_009057_20190903_20190917_01_T1_B4.TIF',
 'src/Landsat/LC08_L1TP_009057_20190903_20190917_01_T1_B5.TIF']

In [43]:
#
# Rutina para medicación de tiempo empleado.
now = datetime.now()
timestamp = datetime.timestamp(now)

#
# Inicialización de rutas de trabajo
raster_out_path = "src/Landsat/earthpy/Landsat_Out.tif"
arr, arr_meta = es.stack(images, raster_out_path)

#
# Rutina para medicación de tiempo empleado.
now = datetime.now()
timestamp2 = datetime.timestamp(now)
print("Layer Stack Finalizado.\nDuración de la actividad: {:.2f} segundos" .format(timestamp2-timestamp))

Layer Stack Finalizado.
Duración de la actividad: 287.24 segundos


In [47]:
arr_meta

{'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 7591, 'height': 7751, 'count': 4, 'crs': CRS.from_epsg(32618), 'transform': Affine(30.0, 0.0, 275385.0,
       0.0, -30.0, 595515.0), 'tiled': False, 'interleave': 'pixel'}